# Covid Developement in Australia

Team Members:-
    1. Akshaya Raja  , 219203265
    2. Soumith Potturi, 219203031
    3. Karan Jagad , 219203187

PROBLEM STATEMENT - 
    To analyze the development of confirmed COVID-19 infections2 from 2/1/20 to 3/21/20 in
Australia and predict possible infection rates for 3/22/20 and 3/23/20. Compare your predictions with the actual infection rates.

# Analysis

In [1]:
import findspark 
findspark.init()
findspark.find()

'C:\\spark-3.0.0-bin-hadoop2.7'

In [2]:
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import col,sum
from pyspark.ml.feature import VectorAssembler
from pyspark.sql import SQLContext
import pandas as pd
from pyspark.ml.regression import LinearRegression

In [3]:
sc=SparkContext("local","assignment")
sc.setLogLevel("ERROR")
spark=SparkSession(sc)

In [4]:
#Importing data to spark

covid_data=spark.read.option("header","true").option("inferSchema","true").csv("E:\BIG DATA\COVID-19Prediction\ConfirmedCases.txt")
covid_data.show()

+----------------+--------------------+--------+---------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+
|  Province/State|      Country/Region|     Lat|     Long|1/22/20|1/23/20|1/24/20|1/25/20|1/26/20|1/27/20|1/28/20|1/29/20|1/30/20|1/31/20|2/1/20|2/2/20|2/3/20|2/4/20|2/5/20|2/6/20|2/7/20|2/8/20|2/9/20|2/10/20|2/11/20|2/12/20|2/13/20|2/14/20|2/15/20|2/16/20|2/17/20|2/18/20|2/19/20|2/20/20|2/21/20|2/22/20|2/23/20|2/24/20|2/25/20|2/26/20|2/27/20|2/28/20|2/29/20|3/1/20|3/2/20|3/3/20|3/4/20|3/5/20|3/6/20|3/7/20|3/8/20|3/9/20|3/10/20|3/11/20|3/12/20|3/13/20|3/14/20

In [5]:
#Pre-processing data
#Dropping rows with null values
covid_no_null=covid_data.na.drop('any')
covid_no_null.show()

+--------------------+--------------+--------+---------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+
|      Province/State|Country/Region|     Lat|     Long|1/22/20|1/23/20|1/24/20|1/25/20|1/26/20|1/27/20|1/28/20|1/29/20|1/30/20|1/31/20|2/1/20|2/2/20|2/3/20|2/4/20|2/5/20|2/6/20|2/7/20|2/8/20|2/9/20|2/10/20|2/11/20|2/12/20|2/13/20|2/14/20|2/15/20|2/16/20|2/17/20|2/18/20|2/19/20|2/20/20|2/21/20|2/22/20|2/23/20|2/24/20|2/25/20|2/26/20|2/27/20|2/28/20|2/29/20|3/1/20|3/2/20|3/3/20|3/4/20|3/5/20|3/6/20|3/7/20|3/8/20|3/9/20|3/10/20|3/11/20|3/12/20|3/13/20|3/14/20|3/1

In [6]:
#Filtering data related to Australia

covid_data_australia=covid_no_null.filter(col("Country/Region")=="Australia")
covid_data_australia.show()

+--------------------+--------------+--------+--------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+
|      Province/State|Country/Region|     Lat|    Long|1/22/20|1/23/20|1/24/20|1/25/20|1/26/20|1/27/20|1/28/20|1/29/20|1/30/20|1/31/20|2/1/20|2/2/20|2/3/20|2/4/20|2/5/20|2/6/20|2/7/20|2/8/20|2/9/20|2/10/20|2/11/20|2/12/20|2/13/20|2/14/20|2/15/20|2/16/20|2/17/20|2/18/20|2/19/20|2/20/20|2/21/20|2/22/20|2/23/20|2/24/20|2/25/20|2/26/20|2/27/20|2/28/20|2/29/20|3/1/20|3/2/20|3/3/20|3/4/20|3/5/20|3/6/20|3/7/20|3/8/20|3/9/20|3/10/20|3/11/20|3/12/20|3/13/20|3/14/20|3/15/

In [7]:
#Dropping unneccessary columns

covid_data_aust=covid_data_australia.drop('Province/State','Lat','Long')
covid_data_aust.show()

+--------------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+
|Country/Region|1/22/20|1/23/20|1/24/20|1/25/20|1/26/20|1/27/20|1/28/20|1/29/20|1/30/20|1/31/20|2/1/20|2/2/20|2/3/20|2/4/20|2/5/20|2/6/20|2/7/20|2/8/20|2/9/20|2/10/20|2/11/20|2/12/20|2/13/20|2/14/20|2/15/20|2/16/20|2/17/20|2/18/20|2/19/20|2/20/20|2/21/20|2/22/20|2/23/20|2/24/20|2/25/20|2/26/20|2/27/20|2/28/20|2/29/20|3/1/20|3/2/20|3/3/20|3/4/20|3/5/20|3/6/20|3/7/20|3/8/20|3/9/20|3/10/20|3/11/20|3/12/20|3/13/20|3/14/20|3/15/20|3/16/20|3/17/20|3/18/20|3/19/20|3/20/20|3/21/20|3/22/20|3/23/20|
+---------

In [8]:
#Extracting data for the dates required for analysis

col_names=covid_data_aust.columns
col_names=col_names[11:]
covid_data_reqd=covid_data_aust.select([c for  c in covid_data_aust.columns if ((c in col_names) | (c=="Country/Region"))])
covid_data_reqd.show()

+--------------+------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+
|Country/Region|2/1/20|2/2/20|2/3/20|2/4/20|2/5/20|2/6/20|2/7/20|2/8/20|2/9/20|2/10/20|2/11/20|2/12/20|2/13/20|2/14/20|2/15/20|2/16/20|2/17/20|2/18/20|2/19/20|2/20/20|2/21/20|2/22/20|2/23/20|2/24/20|2/25/20|2/26/20|2/27/20|2/28/20|2/29/20|3/1/20|3/2/20|3/3/20|3/4/20|3/5/20|3/6/20|3/7/20|3/8/20|3/9/20|3/10/20|3/11/20|3/12/20|3/13/20|3/14/20|3/15/20|3/16/20|3/17/20|3/18/20|3/19/20|3/20/20|3/21/20|3/22/20|3/23/20|
+--------------+------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+---

In [9]:
#Merging all States

covid_data_reqd=covid_data_reqd.groupBy().sum()
covid_data_reqd.show()

+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+
|sum(2/1/20)|sum(2/2/20)|sum(2/3/20)|sum(2/4/20)|sum(2/5/20)|sum(2/6/20)|sum(2/7/20)|sum(2/8/20)|sum(2/9/20)|sum(2/10/20)|sum(2/11/20)|sum(2/12/20)|sum(2/13/20)|sum(2/14/20)|sum(2/15/20)|sum(2/16/20)|sum(2/17/20)|sum(2/18/20)|sum(2/19/20)|sum(2/20/20)|sum(2/21/20)|sum(2/22/20)|sum(2/23/20)|sum(2/24/20)|sum(2/25/20)|sum(2/26/20)|sum(2/27/2

In [10]:
analysis_data=covid_data_reqd.toPandas()
analysis_data

,sum(2/1/20),sum(2/2/20),sum(2/3/20),sum(2/4/20),sum(2/5/20),sum(2/6/20),sum(2/7/20),sum(2/8/20),sum(2/9/20),sum(2/10/20),...,sum(3/14/20),sum(3/15/20),sum(3/16/20),sum(3/17/20),sum(3/18/20),sum(3/19/20),sum(3/20/20),sum(3/21/20),sum(3/22/20),sum(3/23/20)
0,12,12,12,13,13,14,15,15,15,15,...,250,297,377,452,568,681,791,1071,1314,1314


In [11]:
analysis_data = analysis_data.transpose()
day=[x for x in range(len(covid_data_reqd.columns)) ]
analysis_data['Day']=day
analysis_data.rename(columns={0:"Count"},inplace=True)
analysis_data

,Count,Day
sum(2/1/20),12,0
sum(2/2/20),12,1
sum(2/3/20),12,2
sum(2/4/20),13,3
sum(2/5/20),13,4
sum(2/6/20),14,5
sum(2/7/20),15,6
sum(2/8/20),15,7
sum(2/9/20),15,8
sum(2/10/20),15,9


In [23]:
#Function to form vector column of input data

def prepareForML(analysis_data):
    sqlContext = SQLContext(sc)
    vectorAssembler = VectorAssembler(inputCols = ['Day'], outputCol = 'features')
    spark_data = sqlContext.createDataFrame(analysis_data)
    vector_df = vectorAssembler.transform(spark_data)
    # Days 50 and 51 has to be predicted. So splitting the data into training and testing df.
    train_df=vector_df.where(col('Day')<50)
    predict_df=vector_df.where(col('Day')>49)
    train_df = train_df.select(['features', 'Count'])
    predict_df = predict_df.select(['features', 'Count'])
    
    return train_df,predict_df

train_df,predict_df=prepareForML(analysis_data)

In [24]:
train_df.show()
predict_df.show()

+--------+-----+
|features|Count|
+--------+-----+
|   [0.0]|   12|
|   [1.0]|   12|
|   [2.0]|   12|
|   [3.0]|   13|
|   [4.0]|   13|
|   [5.0]|   14|
|   [6.0]|   15|
|   [7.0]|   15|
|   [8.0]|   15|
|   [9.0]|   15|
|  [10.0]|   15|
|  [11.0]|   15|
|  [12.0]|   15|
|  [13.0]|   15|
|  [14.0]|   15|
|  [15.0]|   15|
|  [16.0]|   15|
|  [17.0]|   15|
|  [18.0]|   15|
|  [19.0]|   15|
+--------+-----+
only showing top 20 rows

+--------+-----+
|features|Count|
+--------+-----+
|  [50.0]| 1314|
|  [51.0]| 1314|
+--------+-----+



In [26]:
def linearReg(training_df):
    lr = LinearRegression(featuresCol = 'features', labelCol='Count', maxIter=10, regParam=0.3, elasticNetParam=0.8)
    lr_model = lr.fit(training_df)
    
    return lr_model

In [27]:
def predict(test_df, lr_model):
    lr_predictions = lr_model.transform(test_df)
    lr_predictions.select("prediction","Count","features").show()
    
    return lr_predictions

In [29]:
#Training the model
lr_model= linearReg(train_df)

#Evaluating the model
prediction=predict(predict_df,lr_model)

prediction.withColumn('Difference', col('Count')-col('prediction')).show()

+------------------+-----+--------+
|        prediction|Count|features|
+------------------+-----+--------+
| 384.1298169541139| 1314|  [50.0]|
|394.46510389349095| 1314|  [51.0]|
+------------------+-----+--------+

+--------+-----+------------------+----------------+
|features|Count|        prediction|      Difference|
+--------+-----+------------------+----------------+
|  [50.0]| 1314| 384.1298169541139|929.870183045886|
|  [51.0]| 1314|394.46510389349095|919.534896106509|
+--------+-----+------------------+----------------+



The difference between actual and predicted value is very high. This maybe because the values in the initial days were very less and there is change in the infection rate after a month. 

It would be a better model if data during this period is taken for training the model.

# Changing the training data

In [30]:
#extracting new dataset from previous
new_data=analysis_data[analysis_data['Day']>36]

#preparing data for Machine Learning
train_df_1,predict_df_1=prepareForML(new_data)


In [31]:
train_df_1.show()
predict_df_1.show()

+--------+-----+
|features|Count|
+--------+-----+
|  [37.0]|   91|
|  [38.0]|  107|
|  [39.0]|  128|
|  [40.0]|  128|
|  [41.0]|  200|
|  [42.0]|  250|
|  [43.0]|  297|
|  [44.0]|  377|
|  [45.0]|  452|
|  [46.0]|  568|
|  [47.0]|  681|
|  [48.0]|  791|
|  [49.0]| 1071|
+--------+-----+

+--------+-----+
|features|Count|
+--------+-----+
|  [50.0]| 1314|
|  [51.0]| 1314|
+--------+-----+



In [33]:
#Training the Linear Regression model
lr_model_new=linearReg(train_df_1)

#Evaluating the model 
prediction_1 = predict(predict_df_1, lr_model_new)

+-----------------+-----+--------+
|       prediction|Count|features|
+-----------------+-----+--------+
|912.7142928352037| 1314|  [50.0]|
| 986.607543460013| 1314|  [51.0]|
+-----------------+-----+--------+



In [34]:
prediction_1.withColumn('Difference', col('Count')-col('prediction')).show()

+--------+-----+-----------------+------------------+
|features|Count|       prediction|        Difference|
+--------+-----+-----------------+------------------+
|  [50.0]| 1314|912.7142928352037|401.28570716479635|
|  [51.0]| 1314| 986.607543460013|327.39245653998705|
+--------+-----+-----------------+------------------+



The prediction done using data where the number of cofirmed cases is constantky increasing gives a better predicted value as it the linear regression model is trained well with that data. 

The difference between Actual and Predicted is comparatively less in this case.  

Furthermore, last 6 days data can be considered for better predictions.

# Considering last 6 days data 

In [38]:
#extracting new dataset from previous
latest_data=analysis_data[analysis_data['Day']>43]

#preparing data for Machine Learning
train_df_2,predict_df_2=prepareForML(latest_data)

train_df_2.show()
predict_df_2.show()

#Training Linear Regression model
lr_model_latest=linearReg(train_df_2)

#Evaluating the model 
prediction_2 = predict(predict_df_2, lr_model_latest)

prediction_2.withColumn('Difference', col('Count')-col('prediction')).show()

+--------+-----+
|features|Count|
+--------+-----+
|  [44.0]|  377|
|  [45.0]|  452|
|  [46.0]|  568|
|  [47.0]|  681|
|  [48.0]|  791|
|  [49.0]| 1071|
+--------+-----+

+--------+-----+
|features|Count|
+--------+-----+
|  [50.0]| 1314|
|  [51.0]| 1314|
+--------+-----+

+------------------+-----+--------+
|        prediction|Count|features|
+------------------+-----+--------+
|1116.0551370916946| 1314|  [50.0]|
|1247.3089857845598| 1314|  [51.0]|
+------------------+-----+--------+

+--------+-----+------------------+------------------+
|features|Count|        prediction|        Difference|
+--------+-----+------------------+------------------+
|  [50.0]| 1314|1116.0551370916946|197.94486290830537|
|  [51.0]| 1314|1247.3089857845598| 66.69101421544019|
+--------+-----+------------------+------------------+



# Conclusion

We have received prediction value close to the actual value when the recent dataset is considered. 

In [ ]:
sc.stop()